# Import statements and magic

In [1]:
# Core libraries
import pandas as pd
import numpy as np

# Network analysis
import networkx as nx

# Plotting capability
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load core datasets

The [TfL Tube map](https://tfl.gov.uk/maps/track/tube) is becoming increasingly dense, as more and more lines and stations are added to it.  By the time I achieve anything with this graph analysis, there may be more tweaking to do because of Crossrail!

I have tried to find a graph online that is set up for basic analysis, which accounts for:
* Interchanges within station
* Interchanges between stations, i.e. OSIs and other short walking routes.
* The latest TfL network, especially all the current London Overground routes.

There are interactive tools (e.g. Oliver O'Brien's [Tube Creature](http://tubecreature.com/)) and Google Maps which cover those three factors, but I have not found a workable dataset that can be used for my own graph analysis.

[Mark Dunne's Tube graph project](http://markdunne.github.io/2016/04/10/The-London-Tube-as-a-Graph/) is the best graph analysis I have come across, which used data from [Nicola Greco](https://github.com/nicola).  However, the data do not cover the three elements specified above.

-----
__The two core challenges I want to tackle in my analysis:__
* Calculate the shortest path between two stations.
* Perform graph analysis / link analysis on the network.
-----

I have created my own dataset to include all the Overground routes and added my own interchange edges.  There are about 120 OSI and walking routes in my dataset (fairly even split between the two).

For my dataset, I have also designed an ID system that includes information about all the stations: e.g. Piccadilly Circus has ID 71034, where '10' shows that it is in Zone 1.  This numbering system has been useful for producing pivot tables and bringing in an element of data verification.

Each node in the network acts as a platform, rather than a station itself.  The node notation is in the 'station [line]' format, e.g.:
* 'Baker Street [Bakerloo]' node represents a Bakerloo platform (note the square brackets) at Baker Street station.
* 'Baker Street' node represents Baker Street station (as if street level or ticket hall).

The neighbours of a station node include all its platforms and also nearby stations.

In [2]:
connections = pd.read_csv('connections-interchange.csv', index_col=0)

In [3]:
connections.head()

,station1,station2,service,time,node1_long,node2_long,service_long,comment,within_station,plat1,plat2,plat1serv,plat2serv
urn_old,,,,,,,,,,,,,
1.0,71004,01009,702,1.0,Baker Street [Bakerloo],Marylebone [Bakerloo],Bakerloo,NaN,False,NaN,NaN,NaN,NaN
2.0,71004,71037,702,2.0,Baker Street [Bakerloo],Regent's Park [Bakerloo],Bakerloo,NaN,False,NaN,NaN,NaN,NaN
3.0,01003,71014,702,1.0,Charing Cross [Bakerloo],Embankment [Bakerloo],Bakerloo,NaN,False,NaN,NaN,NaN,NaN
4.0,01003,71034,702,2.0,Charing Cross [Bakerloo],Piccadilly Circus [Bakerloo],Bakerloo,NaN,False,NaN,NaN,NaN,NaN
5.0,71012,01009,702,2.0,Edgware Road (Bakerloo) [Bakerloo],Marylebone [Bakerloo],Bakerloo,NaN,False,NaN,NaN,NaN,NaN


In [4]:
print("Number of connections in graph:", len(connections))

Number of connections in graph: 1441


# Journey times

Set up a graph with the weights as the journey times (minutes).

In [5]:
graph_times = nx.Graph()

for connection_id, connection in connections.iterrows():
    service_name = connection['service_long'] # e.g. Bakerloo
    node1_name = connection['node1_long'] # e.g. Baker Street [Bakerloo]
    node2_name = connection['node2_long'] # e.g. Marylebone [Bakerloo]

    # Transform time between platforms into weights
    graph_times.add_edge(node1_name, node2_name, weight = connection['time'])

Inspect the graph nodes.

In [6]:
print(graph_times.neighbors("Paddington (Bakerloo)"),
      graph_times.neighbors("Paddington (H&C)"),
      graph_times.neighbors("Paddington"), sep='\n')

['Paddington', 'Paddington (Bakerloo) [Bakerloo]', 'Paddington (Bakerloo) [Circle]']
['Paddington', 'Paddington (H&C) [Circle]', 'Paddington (H&C) [District]', 'Paddington (H&C) [Hammersmith & City]']
['Lancaster Gate', 'Marylebone', 'Paddington (Bakerloo)', 'Paddington (H&C)']


As can be seen above, there are two separate Undeground stations at Paddington:
* Bakerloo station: Bakerloo and Circle services.
* H&C station: H&C, Circle and District services.

Each station has the 'Paddington' node as a neighbour: i.e. the National Rail concourse.  An interchange between the Underground stations involves traversing the National Rail concourse.

Now let's create a function to obtain the __fastest route__ and see how it is working.

In [7]:
def fastest_route(start, end):
    """
    Return the fastest path between the 'start' and 'end' points.
    Each station and interchange is printed, along with the journey time.
    
    Tip: use "" when calling the function, as escape characters may be needed with ''.
    """
    journey_path = nx.shortest_path(graph_times, start, end, weight='weight')
    journey_time = nx.shortest_path_length(graph_times, start, end, weight='weight')
    print('\nJOURNEY:', *journey_path, sep='\n\t')
    print('\nJOURNEY TIME:', journey_time, 'minutes')

In [8]:
# Google Maps has Paddington concourse -> Notting Hill Gate as 5min journey
fastest_route("Paddington", "Notting Hill Gate [Circle]") # start: National Rail
fastest_route("Paddington", "Notting Hill Gate")
fastest_route("Paddington (H&C)", "Notting Hill Gate") # start: north end of Paddington


JOURNEY:
	Paddington
	Paddington (Bakerloo)
	Paddington (Bakerloo) [Circle]
	Bayswater [Circle]
	Notting Hill Gate [Circle]

JOURNEY TIME: 9.0 minutes

JOURNEY:
	Paddington
	Paddington (Bakerloo)
	Paddington (Bakerloo) [Circle]
	Bayswater [Circle]
	Notting Hill Gate [Circle]
	Notting Hill Gate

JOURNEY TIME: 11.0 minutes

JOURNEY:
	Paddington (H&C)
	Paddington
	Paddington (Bakerloo)
	Paddington (Bakerloo) [Circle]
	Bayswater [Circle]
	Notting Hill Gate [Circle]
	Notting Hill Gate

JOURNEY TIME: 14.0 minutes


In [9]:
fastest_route("Marylebone", "Notting Hill Gate")


JOURNEY:
	Marylebone
	Edgware Road (Circle)
	Edgware Road (Circle) [Circle]
	Paddington (Bakerloo) [Circle]
	Bayswater [Circle]
	Notting Hill Gate [Circle]
	Notting Hill Gate

JOURNEY TIME: 19.0 minutes


In [10]:
fastest_route("Paddington (H&C) [Circle]", "Paddington (Bakerloo) [Circle]")
# The two Circle services at Edgware Rd are at different stations - this journey is not right


JOURNEY:
	Paddington (H&C) [Circle]
	Edgware Road (Circle) [Circle]
	Paddington (Bakerloo) [Circle]

JOURNEY TIME: 6.0 minutes


In [11]:
fastest_route("Angel", "Mornington Crescent")
# Interchange at Euston between Northern platforms not working because the nodes have the same name


JOURNEY:
	Angel
	Angel [Northern]
	King's Cross St. Pancras [Northern]
	Euston [Northern]
	Mornington Crescent [Northern]
	Mornington Crescent

JOURNEY TIME: 12.0 minutes


## West Hampstead to West Ruislip

There are three separate stations in the West Hampstead area (Underground, Overground and Thameslink) so that opens up at least three different routes to West Ruislip, which can take approx. 50 minutes.

The journey is a useful test for the data pipeline.

In [12]:
# 'West Hampstead' is the Overground station
fastest_route("West Hampstead", "West Ruislip")


JOURNEY:
	West Hampstead
	West Hampstead Underground
	West Hampstead Underground [Jubilee]
	Finchley Road [Jubilee]
	Finchley Road [Metropolitan]
	Wembley Park [Metropolitan]
	Preston Road [Metropolitan]
	Northwick Park [Metropolitan]
	Harrow-on-the-Hill [Metropolitan]
	West Harrow [Metropolitan]
	Rayners Lane [Metropolitan]
	Eastcote [Metropolitan]
	Ruislip Manor [Metropolitan]
	Ruislip [Metropolitan]
	Ickenham [Metropolitan]
	Ickenham
	West Ruislip

JOURNEY TIME: 49.0 minutes


There are three different routes that could take a similar amount of time (approx. 50 minutes, +/- a few minutes):
* Take Jubilee line to Bond Street, then take the Central line direct to West Ruislip.
* Get on the Metropolitan line (Finchley Road), leave at Ickenham and then walk from there to West Ruislip.  That walking route is an OSI on the network.
* Take Overground service to Shepherd's Bush, then take the Central line direct to West Ruislip.

On Google Maps, the journey time for one route can have variation by a few minutes, depending on the actual interchange time (e.g. the wait for Overground train).

With the interchange data, the fastest route from West Hampstead Overground to West Ruislip takes 49.0 minutes.  The new model is suggesting a walk to the Underground station, instead of taking the Overground to Shepherd’s Bush and taking the Central line from there:

In [13]:
fastest_route("West Hampstead Underground", "West Ruislip") # Jubilee > Metropolitan
fastest_route("Swiss Cottage [Jubilee]", "West Ruislip") # Jubilee > Central
fastest_route("Brondesbury [NLL]", "West Ruislip") # Overground > Central


JOURNEY:
	West Hampstead Underground
	West Hampstead Underground [Jubilee]
	Finchley Road [Jubilee]
	Finchley Road [Metropolitan]
	Wembley Park [Metropolitan]
	Preston Road [Metropolitan]
	Northwick Park [Metropolitan]
	Harrow-on-the-Hill [Metropolitan]
	West Harrow [Metropolitan]
	Rayners Lane [Metropolitan]
	Eastcote [Metropolitan]
	Ruislip Manor [Metropolitan]
	Ruislip [Metropolitan]
	Ickenham [Metropolitan]
	Ickenham
	West Ruislip

JOURNEY TIME: 48.0 minutes

JOURNEY:
	Swiss Cottage [Jubilee]
	St. John's Wood [Jubilee]
	Baker Street [Jubilee]
	Bond Street [Jubilee]
	Bond Street [Central]
	Marble Arch [Central]
	Lancaster Gate [Central]
	Queensway [Central]
	Notting Hill Gate [Central]
	Holland Park [Central]
	Shepherd's Bush (Central) [Central]
	White City [Central]
	East Acton [Central]
	North Acton [Central]
	Hanger Lane [Central]
	Perivale [Central]
	Greenford [Central]
	Northolt [Central]
	South Ruislip [Central]
	Ruislip Gardens [Central]
	West Ruislip [Central]
	West Ruislip

The route via the Jubilee line and Central line is not far off on journey time though.  It takes 47.0 minutes from Swiss Cottage’s Jubilee line platform to West Ruislip. In total, the journey from West Hampstead Underground to West Ruislip via Jubilee and Central lines is 51.0 minutes.

In [14]:
fastest_route("West Hampstead Underground", "Swiss Cottage [Jubilee]")


JOURNEY:
	West Hampstead Underground
	West Hampstead Underground [Jubilee]
	Finchley Road [Jubilee]
	Swiss Cottage [Jubilee]

JOURNEY TIME: 4.0 minutes


# Graph analysis

## Centrality

Set up the centrality algorithm for the network, starting platform by platform.

In [15]:
centrality_dict = nx.closeness_centrality(graph_times)
centrality_p = pd.DataFrame.from_dict(centrality_dict, 'index').reset_index() # dataframe
centrality_p.columns = ['node', 'value'] # set column names
centrality_p.sort_values('value', ascending=False).head()

,node,value
369,King's Cross St. Pancras [Victoria],0.094824
261,King's Cross St. Pancras [Northern],0.093727
246,King's Cross St. Pancras [Metropolitan],0.093662
95,King's Cross St. Pancras [Circle],0.093494
368,Euston [Victoria],0.093457


Aggregate the nodes by station.

In [16]:
# Temporary frame: station names to match to the rows of previous frame
centrality_temp = pd.DataFrame(centrality_p['node'].str.split(' \[').str.get(0)) # get station
centrality_temp.columns = ['station']
centrality_temp.head()

,station
0,Baker Street
1,Marylebone
2,Regent's Park
3,Charing Cross
4,Embankment


In [17]:
centrality_p2 = centrality_p.join(centrality_temp) # join the frames
centrality_s = centrality_p2.groupby('station')[['value']].sum().reset_index() # get aggregation
centrality_s.columns = ['station', 'value (agg)'] # unbreak ugly header formatting
centrality_s.sort_values('value (agg)', ascending=False).head(25)

,station,value (agg)
206,King's Cross St. Pancras,0.650959
13,Baker Street,0.541744
221,Liverpool Street,0.504331
236,Moorgate,0.433034
15,Bank,0.431411
380,Waterloo,0.418083
120,Embankment,0.414187
340,Stratford,0.367898
124,Euston,0.366513
408,Willesden Junction,0.365907


Stations combined that go firmly in the top 5:
* Bank and Monument stations (Bank is the more influential station)
* Paddington (both Underground stations have similar influence)

Main hubs outside Zone 1 (from centrality algorithm):
* Stratford
* Willesden Junction
* Highbury & Islington
* West Ham

In [18]:
centrality_s.sort_values('value (agg)', ascending=False).tail(25)

,station,value (agg)
121,Emerson Park,0.072341
293,Romford,0.069800
116,Elephant and Castle Rail,0.069159
341,Stratford High Street,0.067871
160,Harringay,0.064677
282,Queens Road Peckham,0.063348
266,Peckham Rye,0.061548
99,Denmark Hill,0.060258
374,Wandsworth Road,0.060173
133,Forest Gate,0.055215


## PageRank

Set up the centrality algorithm for the network, starting platform by platform.

In [19]:
graph_weights = nx.Graph()

for connection_id, connection in connections.iterrows():
    service_name = connection['service_long']
    node1_name = connection['node1_long']
    node2_name = connection['node2_long']

    # Transform time between platforms into weights
    graph_weights.add_edge(node1_name, node2_name, weight = 1.0 / connection['time'])

In [20]:
pagerank_dict = nx.pagerank_numpy(graph_weights, weight='weight')
pagerank_p = pd.DataFrame.from_dict(pagerank_dict, orient='index').reset_index()
pagerank_p.columns = ['node', 'pagerank']
pagerank_p.sort_values('pagerank', ascending=False).head(25)

,node,pagerank
283,Camden Town [Northern],0.002242
140,Earl's Court [District],0.002233
513,Poplar [DLR],0.002174
509,Stratford [DLR],0.002103
454,Canada Water [ELL],0.002068
504,Canning Town [DLR],0.001929
461,Sydenham [ELL],0.001920
521,Beckton Park [DLR],0.001904
302,West Finchley [Northern],0.001857
284,Euston [Northern],0.001853


Aggregate the nodes by station.

In [21]:
# Temporary frame: station names to match to the rows of previous frame
pagerank_temp = pd.DataFrame(pagerank_p['node'].str.split(' \[').str.get(0)) # get station
pagerank_temp.columns = ['station']
pagerank_temp.head()

,station
0,Baker Street
1,Marylebone
2,Regent's Park
3,Charing Cross
4,Embankment


In [22]:
pagerank_p2 = pagerank_p.join(pagerank_temp) # join the frames
pagerank_s = pagerank_p2.groupby('station')[['pagerank']].sum().reset_index() # get aggregation
pagerank_s.columns = ['station', 'value (agg)'] # unbreak ugly header formatting
pagerank_s.sort_values('value (agg)', ascending=False).head(25)

,station,value (agg)
206,King's Cross St. Pancras,0.009853
13,Baker Street,0.007427
120,Embankment,0.007008
340,Stratford,0.006404
221,Liverpool Street,0.006252
15,Bank,0.006213
391,West Ham,0.006131
16,Barbican,0.006037
408,Willesden Junction,0.005856
232,Mile End,0.005204


Main hubs outside Zone 1 (from PageRank):
* Stratford
* Willesden Junction
* Mile End
* Barking
* Acton Town
* Whitechapel

In [23]:
pagerank_s.sort_values('value (agg)', ascending=False).tail(25)

,station,value (agg)
196,Kent House,0.001044
406,Whyteleafe,0.001044
367,Upper Warlingham,0.001044
64,Catford,0.001044
65,Catford Bridge,0.001044
42,Bromley South,0.001044
41,Bromley North,0.001044
165,Hatton Cross,0.000982
99,Denmark Hill,0.000916
282,Queens Road Peckham,0.000886


## Wrap-up

The most influential stations in the network, from __both algorithms__, in a rough order of influence:
* King's Cross St. Pancras
* Baker Street
* Liverpool Street
* Other City of London stations, e.g. Moorgate, Bank & Monument, Barbican
* Embankment
* Stratford
* Willesden Junction
* Waterloo

Nearby stations that are also prominent:
* Euston and Euston Square
* Paddington's two Underground stations

Some major stations like Clapham Junction, Victoria and Vauxhall are not appearing high as the network does not account for National Rail routes.


Stations that have relatively high influence under the __centrality algorithm:__
* Oxford Circus
* Green Park
* Highbury & Islington

Stations that have relatively high influence under the __PageRank algorithm:__
* Interchange stations in east London on H&C line: e.g. Mile End, West Ham
* Interchange stations in central London on Circle line: e.g. South Kensington

# Retrospective

* __What has worked well so far?__
    * The network analysis seems intuitive and is an improvement on the initial analysis (e.g. no longer focused on small stations such as Barbican).
    * The Excel workbook with all the data has been quick to edit when significant changes have needed to be made.
    * Some functionality has been straightforward to implement, e.g. shortest path function.
    * Shortest path function results have been improved significantly due to the addition of interchange data to the dataset.


* __What can be improved?__
    * Shortest path function is not working well for a few stations that have interchanges for the same service (e.g. between Edgware Road's Circle platforms; Camden Town's Northern platforms), due to how the nodes are structured.
    * Data visualisation could add much more to the analysis.
    * Other metropolitan rail services could be added to the network (e.g. Thameslink).